# Fresh Prompt
- [freshllms/freshqa](https://github.com/freshllms/freshqa)

## セットアップ

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)

openai_client = OpenAI(
  api_key=os.environ["OPENAI_API_KEY"],
)

In [2]:
import pytz
import datetime

current_date = datetime.datetime.now(
    pytz.timezone('Asia/Tokyo')
).strftime("%B %d, %Y")
print(current_date)

November 15, 2023


In [5]:
temperature = 0.0
max_tokens = 256

def chat_completions(prompt):
    response = openai_client.chat.completions.create(
        model="gpt-4-1106-preview",
        temperature=0.0,
        max_tokens=256,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant. Answer as concisely as"
                    f" possible. Knowledge cutoff: {current_date}."
                ),
            },
            {"role": "user", "content": "What's today's date?"},
            {
                "role": "assistant",
                "content": f"Today is {current_date} in Pacific Standard Time.",
            },
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content


In [4]:
from serpapi import GoogleSearch

def call_search_engine(query):
  params = {
    "q": query,
    "hl": "ja",
    "gl": "jp",
    "google_domain": "google.com",
    "api_key": os.environ["SERPAPI_API_KEY"],
  }

  search = GoogleSearch(params)
  return search.get_dict()

In [6]:
import dateutil
import re

def is_date(string, fuzzy=False):
  # Parse a string into a date and check its validity
  try:
      dateutil.parser.parse(string, fuzzy=fuzzy)
      return True
  except ValueError:
      return False


def format_date(d):
  # Standardize the date format for each search result
  date = dateutil.parser.parse(current_date, fuzzy=True).strftime("%b %d, %Y")
  if d is None:
    return None

  for t in ["second", "minute", "hour"]:
    if f"{t} ago" in d or f"{t}s ago" in d:
      return date

  t = "day"
  if f"{t} ago" in d or f"{t}s ago" in d:
    n_days = int(re.search("(\d+) days? ago", d).group(1))
    return (
        datetime.datetime.strptime(date, "%b %d, %Y")
        - datetime.timedelta(days=n_days)
    ).strftime("%b %d, %Y")

  try:
    return dateutil.parser.parse(d, fuzzy=True).strftime("%b %d, %Y")
  except ValueError:
    for x in d.split():
      if is_date(x):
        return dateutil.parser.parse(x, fuzzy=True).strftime("%b %d, %Y")


def extract_source_webpage(link):
  # Extract source webpage
  return (
      link.strip()
      .replace("https://www.", "")
      .replace("http://www.", "")
      .replace("https://", "")
      .replace("http://", "")
      .split("/")[0]
  )


def simplify_displayed_link(displayed_link):
  # Simplify displayed link
  if displayed_link is None:
    return None
  return extract_source_webpage(displayed_link.split(' › ')[0])

In [7]:
def format_search_results(search_data, title_field=None, highlight_field=None):
  # Standardize search results as shown in Figure 3 (left) in the paper
  field = 'snippet_highlighted_words'
  if field in search_data and isinstance(search_data[field], list):
    search_data[field] = ' | '.join(search_data[field])

  field = 'displayed_link'
  if field in search_data:
    search_data[field] = simplify_displayed_link(search_data[field])

  # edge case 1
  if search_data.get('type') == 'local_time':
    source = search_data.get('displayed_link')
    date = format_date(search_data.get('date'))
    title = search_data.get('title')

    snippet = search_data.get('snippet')
    if snippet is None and 'result' in search_data:
      if 'extensions' in search_data and isinstance(
          search_data['extensions'], list
      ):
        snippet = '\n\t'.join(
            [search_data['result']] + search_data['extensions']
        )
      else:
        snippet = search_data['result']

    highlight = search_data.get('snippet_highlighted_words')
    if highlight is None and 'result' in search_data:
      highlight = search_data['result']

  # edge case 2
  elif 'type' in search_data and search_data['type'] == 'population_result':
    source = search_data.get('displayed_link')
    if source is None and 'sources' in search_data:
      if (
          isinstance(search_data['sources'], list)
          and 'link' in search_data['sources'][0]
      ):
        source = extract_source_webpage(search_data['sources'][0]['link'])

    date = format_date(search_data.get('date'))
    if date is None and 'year' in search_data:
      date = format_date(search_data['year'])

    title = search_data.get('title')

    snippet = search_data.get('snippet')
    if snippet is None and 'population' in search_data:
      if 'place' in search_data:
        snippet = '\n\t'.join(
            [
                f"{search_data['place']} / Population",
            ]
            + [
                search_data['population'],
            ]
        )
      else:
        snippet = search_data['population']

    highlight = search_data.get('snippet_highlighted_words')
    if highlight is None and 'population' in search_data:
      highlight = search_data['population']

  else:
    source = search_data.get('displayed_link')
    date = format_date(search_data.get('date'))
    title = (
        search_data.get('title')
        if title_field is None
        else search_data.get(title_field)
    )
    highlight = (
        search_data.get('snippet_highlighted_words')
        if highlight_field is None
        else search_data.get(highlight_field)
    )
    snippet = search_data.get('snippet', '')

    if 'rich_snippet' in search_data:
      for key in ['top', 'bottom']:
        if (
            key in search_data['rich_snippet']
            and 'extensions' in search_data['rich_snippet'][key]
        ):
          snippet = '\n\t'.join(
              [snippet] + search_data['rich_snippet'][key]['extensions']
          )

    if 'list' in search_data:
      assert isinstance(search_data['list'], list)
      snippet = '\n\t'.join([snippet] + search_data['list'])

    if 'contents' in search_data and 'table' in search_data['contents']:
      tbl = search_data['contents']['table']
      assert isinstance(tbl, list)
      snippet += '\n'
      for row in tbl:
        snippet += f'\n{",".join(row)}'

    if snippet is not None and snippet.strip() == '':
      snippet = None

  return {
      'source': source,
      'date': date,
      'title': title,
      'snippet': snippet,
      'highlight': highlight,
  }

In [8]:
def format_knowledge_graph(search_data):
  # Standardize knowledge graphs as shown in Figure 3 (left) in the paper
  source = None
  if "source" in search_data and "link" in search_data["source"]:
    source = extract_source_webpage(search_data["source"]["link"])

  date = None

  title = None
  if "title" in search_data:
    title = search_data["title"]
    if "type" in search_data:
      title += f"\n\t{search_data['type']}"

  snippet = ""
  for field in search_data:
    if (
        (field not in ["title", "type", "kgmid"])
        and ("_link" not in field)
        and ("_stick" not in field)
        and isinstance(search_data[field], str)
        and not search_data[field].startswith("http")
    ):
      snippet += f"\n\t{field}: {search_data[field]}"

  if snippet.strip() == "":
    snippet = None
  else:
    snippet = snippet.strip()

  highlight = None

  return {
      "source": source,
      "date": date,
      "title": title,
      "snippet": snippet,
      "highlight": highlight,
  }


def format_questions_and_answers(search_data):
  # Standardize questions and answers as shown in Figure 3 (left) in the paper
  source = None
  if "link" in search_data:
    source = extract_source_webpage(search_data["link"])

  date = None

  title = None
  if "question" in search_data:
    title = search_data["question"]

  snippet = None
  if "answer" in search_data:
    snippet = search_data["answer"]

  highlight = None

  return {
      "source": source,
      "date": date,
      "title": title,
      "snippet": snippet,
      "highlight": highlight,
  }



## Demo Prompts

In [9]:
# read from file
with open("./data/freshprompt_demo.txt", "r") as f:
    freshprompt_demo = f.read()

print(freshprompt_demo)

query: What year is considered Albert Einstein's annus mirabilis?

source: philsci-archive.pitt.edu
date: None
title: The Turning Point for Einstein's Annus Mirabilis
snippet: The year 1905 has been called Einstein's annus mirabilis in virtue of three ground-breaking works completed over the span of a few months — the light.
highlight: 1905

source: cantorsparadise.com
date: None
title: Einstein's Miraculous Year: A Summary of the 1905 Annus ...
snippet: These are the four papers that Albert Einstein published in 1905, which are considered to be the foundation of modern physics.
highlight: 1905

source: quora.com
date: None
title: Why is 1905 Einstein's miracle year?
snippet: Thus, 1905 is called Einstein's annus mirabilis, or miracle year, and these papers are called his annus mirabilis papers .
highlight: 1905

source: en.wikipedia.org
date: None
title: Annus mirabilis papers
snippet: The annus mirabilis papers are the four papers that Albert Einstein published in Annalen der Physik 

## freshprompt_format method

In [10]:
question = "今年のルヴァンカップを優勝したサッカーチームはどこですか？"

In [11]:
# modelにgpt-4を利用する場合
num_organic_results = 15
num_related_questions = 3
num_questions_and_answers = 3
num_retrieved_evidences = 15

In [12]:
import pandas as pd

df = pd.DataFrame(columns=['source', 'date', 'title', 'snippet', 'highlight'])

In [13]:
search_data = call_search_engine(question)

In [14]:
search_data

{'search_metadata': {'id': '6553e5a3c56d93c332827067',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/b4ff1bef10e4c0d8/6553e5a3c56d93c332827067.json',
  'created_at': '2023-11-14 21:24:51 UTC',
  'processed_at': '2023-11-14 21:24:51 UTC',
  'google_url': 'https://www.google.com/search?q=%E4%BB%8A%E5%B9%B4%E3%81%AE%E3%83%AB%E3%83%B4%E3%82%A1%E3%83%B3%E3%82%AB%E3%83%83%E3%83%97%E3%82%92%E5%84%AA%E5%8B%9D%E3%81%97%E3%81%9F%E3%82%B5%E3%83%83%E3%82%AB%E3%83%BC%E3%83%81%E3%83%BC%E3%83%A0%E3%81%AF%E3%81%A9%E3%81%93%E3%81%A7%E3%81%99%E3%81%8B%EF%BC%9F&oq=%E4%BB%8A%E5%B9%B4%E3%81%AE%E3%83%AB%E3%83%B4%E3%82%A1%E3%83%B3%E3%82%AB%E3%83%83%E3%83%97%E3%82%92%E5%84%AA%E5%8B%9D%E3%81%97%E3%81%9F%E3%82%B5%E3%83%83%E3%82%AB%E3%83%BC%E3%83%81%E3%83%BC%E3%83%A0%E3%81%AF%E3%81%A9%E3%81%93%E3%81%A7%E3%81%99%E3%81%8B%EF%BC%9F&hl=ja&gl=jp&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/b4ff1bef10e4c0d8/6553e5a3c56d93c332827067.html',
  'total_time_taken': 2

In [15]:
# Organic results
organic_results = [None] * num_organic_results
for k in range(num_organic_results):
    if (
        'organic_results' in search_data
        and len(search_data['organic_results']) > k
    ):
      organic_results[k] = format_search_results(
          search_data['organic_results'][k]
      )
    else:
      organic_results[k] = format_search_results({})

for d in organic_results[::-1]:
   df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)

In [16]:
print(len(organic_results))
organic_results

15


[{'source': 'soccer-king.jp',
  'date': 'Nov 04, 2023',
  'title': 'ルヴァンカップMVPは福岡MF前寛之！ 恩師・長谷部監督 ...',
  'snippet': '『2023JリーグYBCルヴァンカップ』の決勝戦が4日に行われ、アビスパ福岡が2－1で浦和レッズを下して初優勝。この試合で貴重な先制点を記録し、MVPを ...',
  'highlight': 'ルヴァンカップ | 優勝'},
 {'source': 'jleague.jp',
  'date': 'Jun 18, 2023',
  'title': '【公式】YBCルヴァンカップの順位表 - Jリーグ',
  'snippet': '（1）グループステージが終了した時点で、勝点合計の多いチームを上位とし、順位を決定する。ただし、勝点が同じ場合は、以下の順によって順位を決定する ...',
  'highlight': 'した | チーム'},
 {'source': 'detail.chiebukuro.yahoo.co.jp',
  'date': 'Nov 08, 2023',
  'title': 'ルヴァンカップの決勝でアビスパ福岡が優勝しましたが',
  'snippet': 'ルヴァンカップの決勝でアビスパ福岡が優勝しましたが、福岡でもソフトバンクホークスの影に隠れていることが多く厳しい状況が続いていたため感動も ...',
  'highlight': 'ルヴァンカップ | 優勝 | した'},
 {'source': 'sposuru.com',
  'date': 'Aug 09, 2023',
  'title': '【ルヴァンカップ】賞金と歴代優勝チーム一覧 - スポスル',
  'snippet': '昔からのサッカーファンには「ヤマザキナビスコカップ」として知られているルヴァンカップ。 Jリーグの始まりと共にスタートしたこの大会ではさまざまな ...',
  'highlight': 'サッカー | ルヴァンカップ | した'},
 {'source': 'jleague.jp',
  'date': None,
  'title': '歴代優勝クラブ：2022JリーグYBCルヴァンカップ',
  'sni

In [17]:
 # Related questions
related_questions = [None] * num_related_questions
for k in range(num_related_questions):
  if (
      'related_questions' in search_data
      and len(search_data['related_questions']) > k
  ):
    related_questions[k] = format_search_results(
        search_data['related_questions'][k], title_field='question'
    )
  else:
    related_questions[k] = format_search_results({})

for d in related_questions[::-1]:
  df = pd.concat([df, pd.DataFrame([d])], ignore_index=True)


In [18]:
print(len(related_questions))
related_questions

3


[{'source': 'jleague.jp',
  'date': None,
  'title': 'ルヴァンカップ決勝 どこで？',
  'snippet': '決勝の開催日、開催スタジアム、TV放送決定のお知らせ【ルヴァンカップ】 「２０２３ＪリーグYBCルヴァンカップ 決勝」の開催日、開催スタジアムおよびTV放送が下記の通り決定しましたのでお知らせいたします。 決勝は11月4日（土）に国立競技場にて開催いたします。',
  'highlight': None},
 {'source': 'nikkansports.com',
  'date': None,
  'title': 'ルヴァンカップの優勝者は？',
  'snippet': None,
  'highlight': None},
 {'source': 'jleague.jp',
  'date': None,
  'title': 'ルヴァンカップのグループBの順位は？',
  'snippet': None,
  'highlight': None}]

In [19]:
# Knowledge graph
knowledge_graph = None
if 'knowledge_graph' in search_data:
  knowledge_graph = format_knowledge_graph(search_data['knowledge_graph'])
else:
  knowledge_graph = format_knowledge_graph({})
df = pd.concat([df, pd.DataFrame([knowledge_graph])], ignore_index=True)

In [20]:
knowledge_graph

{'source': None,
 'date': None,
 'title': None,
 'snippet': None,
 'highlight': None}

In [21]:
# Answer box
answer_box = None
if 'answer_box' in search_data:
  answer_box = format_search_results(
      search_data['answer_box'], highlight_field='answer'
  )
else:
  answer_box = format_search_results({})
df = pd.concat([df, pd.DataFrame([answer_box])], ignore_index=True)

In [22]:
answer_box

{'source': 'jleague.jp',
 'date': 'Nov 05, 2023',
 'title': '【祝・初タイトル】アビスパ福岡のルヴァンカップ優勝を ...',
 'snippet': '11月4日に国立競技場で２０２３ＪリーグＹＢＣルヴァンカップ決勝が行われ、アビスパ福岡が浦和レッズを下して初優勝に輝きました！',
 'highlight': None}

In [23]:
# Sort by date
df['date'] = df['date'].apply(lambda x: format_date(x))
df['datetime'] = pd.to_datetime(df['date'], errors='coerce')
df = df.sort_values(by='datetime', na_position='first')
df.replace({pd.NaT: None}, inplace=True)
df = df.dropna(how='all')

In [24]:
df.head()

,source,date,title,snippet,highlight,datetime
10,jleague.jp,None,歴代優勝クラブ：2022JリーグYBCルヴァンカップ,2022JリーグYBCルヴァンカップの30周年特設サイト、過去30年の歴代優勝クラブを紹介。...,ルヴァンカップ | 優勝 | サッカー,None
15,jleague.jp,None,ルヴァンカップのグループBの順位は？,None,None,None
16,nikkansports.com,None,ルヴァンカップの優勝者は？,None,None,None
17,jleague.jp,None,ルヴァンカップ決勝 どこで？,決勝の開催日、開催スタジアム、TV放送決定のお知らせ【ルヴァンカップ】 「２０２３ＪリーグY...,None,None
13,jleague.jp,"Jun 18, 2023",【公式】YBCルヴァンカップの順位表 - Jリーグ,（1）グループステージが終了した時点で、勝点合計の多いチームを上位とし、順位を決定する。ただ...,した | チーム,2023-06-18 00:00:00


In [25]:
reasoning_and_answer = (
    "\nPlease check if the question contains a valid premise before"
    " answering.\nanswer: "
)

# Select top_k supporting evidences overall
evidences = []

for _, row in df.tail(num_retrieved_evidences).iterrows():
  evidences.append(
      f"""\n\nsource: {row['source']}\ndate: {row['date']}\ntitle: {row['title']}\nsnippet: {row['snippet']}\nhighlight: {row['highlight']}"""
  )

format_fresh_prompt = ''.join(
        [
            f'\n\n\nquery: {question}',
        ]
        + evidences
    ) + f'\n\nquestion: {question}{reasoning_and_answer}'

In [26]:
print(format_fresh_prompt)




query: 今年のルヴァンカップを優勝したサッカーチームはどこですか？

source: jleague.jp
date: None
title: 歴代優勝クラブ：2022JリーグYBCルヴァンカップ
snippet: 2022JリーグYBCルヴァンカップの30周年特設サイト、過去30年の歴代優勝クラブを紹介。Jリーグ（日本プロサッカーリーグ）試合速報、日程、結果、順位表、写真、 ...
highlight: ルヴァンカップ | 優勝 | サッカー

source: jleague.jp
date: None
title: ルヴァンカップのグループBの順位は？
snippet: None
highlight: None

source: nikkansports.com
date: None
title: ルヴァンカップの優勝者は？
snippet: None
highlight: None

source: jleague.jp
date: None
title: ルヴァンカップ決勝 どこで？
snippet: 決勝の開催日、開催スタジアム、TV放送決定のお知らせ【ルヴァンカップ】 「２０２３ＪリーグYBCルヴァンカップ 決勝」の開催日、開催スタジアムおよびTV放送が下記の通り決定しましたのでお知らせいたします。 決勝は11月4日（土）に国立競技場にて開催いたします。
highlight: None

source: jleague.jp
date: Jun 18, 2023
title: 【公式】YBCルヴァンカップの順位表 - Jリーグ
snippet: （1）グループステージが終了した時点で、勝点合計の多いチームを上位とし、順位を決定する。ただし、勝点が同じ場合は、以下の順によって順位を決定する ...
highlight: した | チーム

source: sposuru.com
date: Aug 09, 2023
title: 【ルヴァンカップ】賞金と歴代優勝チーム一覧 - スポスル
snippet: 昔からのサッカーファンには「ヤマザキナビスコカップ」として知られているルヴァンカップ。 Jリーグの始まりと共にスタートしたこの大会ではさまざまな ...
highlight: サッカー | ルヴァンカップ | した

source: yab.y

In [27]:
fresh_prompt = freshprompt_demo + format_fresh_prompt

In [28]:
print(fresh_prompt)

query: What year is considered Albert Einstein's annus mirabilis?

source: philsci-archive.pitt.edu
date: None
title: The Turning Point for Einstein's Annus Mirabilis
snippet: The year 1905 has been called Einstein's annus mirabilis in virtue of three ground-breaking works completed over the span of a few months — the light.
highlight: 1905

source: cantorsparadise.com
date: None
title: Einstein's Miraculous Year: A Summary of the 1905 Annus ...
snippet: These are the four papers that Albert Einstein published in 1905, which are considered to be the foundation of modern physics.
highlight: 1905

source: quora.com
date: None
title: Why is 1905 Einstein's miracle year?
snippet: Thus, 1905 is called Einstein's annus mirabilis, or miracle year, and these papers are called his annus mirabilis papers .
highlight: 1905

source: en.wikipedia.org
date: None
title: Annus mirabilis papers
snippet: The annus mirabilis papers are the four papers that Albert Einstein published in Annalen der Physik 

In [29]:
answer = chat_completions(fresh_prompt)

In [30]:
print(answer)

2023年のルヴァンカップはアビスパ福岡が優勝しました。
